<h1 align="center"><img align="center" src="https://geoparse.io/graphics/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">All About Points <img src="https://geoparse.io/graphics/point.png" width="10"/> Lines <img src="https://geoparse.io/graphics/line.png" width="40"/> and Polygons <img src="https://geoparse.io/graphics/polygon.png" width="30"/></h3>


#### [HTML](http://geoparse.io/tutorials/karta.html) 
***

# Data Visualization

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to display:

* Points, lines, and polygons.
* Their heatmaps and clusters.
* Their coverage areas using geospatial cells such as H3, S2, and Geohash.
* Geospatial cells on a map based on their indexes.
* OSM roads and buildings using their IDs.
***

In [ ]:
import os
import sys

sys.path.append(os.path.abspath("../geoparse/"))
import warnings

import geopandas as gpd
import pandas as pd

from geoparse import plp

warnings.filterwarnings("ignore")

In [ ]:
# Reload all changed modules every time before executing a new line
%load_ext autoreload
%autoreload 2

## Read from Various File Formats

The [plp](https://github.com/geoparse/geoparse/blob/a30a74db8b1a043c3f1b17946966aa749bc6ae43/geoparse/geoparse.py#L417) (short for point, line, polygon) function in the [Karta class](https://github.com/geoparse/geoparse/blob/a30a74db8b1a043c3f1b17946966aa749bc6ae43/geoparse/geoparse.py#L29) (Swedish for "map") visualizes geometries using either a pandas `DataFrame` or a GeoPandas `GeoDataFrame`. When given a `DataFrame`, `plp` automatically detects columns containing "**lat**" and "**lon**" (case-insensitive) to use as **latitude** and **longitude** for plotting points on the map.

`plp` creates interactive maps with multiple tile layers and supports data visualization from `CSV` (as [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)) and GIS formats like `Shapefile`, `GPKG`, `GeoJSON`, and `GeoParquet` (as [GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html)). 


In [ ]:
df = pd.read_csv("https://geoparse.io/tutorials/data/fatal_crash_great_britain_2023.csv")
df.head()

In [ ]:
len(df)

Available tile layers are:
* **OSM:** A versatile map powered by OpenStreetMap, displaying roads, buildings, and points of interest.
* **Satellite:** Displays high-resolution satellite imagery for real-world context and detailed analysis.
* **Outdoors:** Designed for outdoor activities, featuring trails, elevation contours, and natural landmarks.
* **Dark:** A high-contrast, dark-themed map ideal for vibrant overlays and nighttime visualization.
* **Light:** A clean, minimalistic basemap that highlights overlaid data.

You can switch between different tile layers using the options in the top right corner of the map.

In [ ]:
plp(df)

The `plp` function displays points in blue by default. However, you can change the point color using the `point_color` argument. 

In [ ]:
plp(df, point_color="purple")

For a custom color, use an `RGB` hex code like "#cc5500" for burnt orange.

In [ ]:
plp(df, point_color="#cc5500")

The `plp` function can group points by color based on their values, meaning points with the same value share the same color.
It assigns colors consistently by mapping input column values to a predefined color palette, ensuring a clear and structured visualization.

In [ ]:
plp(df, point_color="speed_limit")

To enhance data interpretation, `plp` allows tooltips that display attribute values when hovering over a point.
The `point_popup` parameter is a dictionary where keys define tooltip labels, and values correspond to column names in the DataFrame.

In [ ]:
plp(df, point_color="speed_limit", point_popup={"Date": "date", "Speed limit": "speed_limit"})

By passing the `heatmap` argument as `True`, `plp` creates a heatmap layer for the points. 
For better visualization, it's recommended to switch to `Dark` mode and uncheck the `Point` layer from the Layer Control menu in the top-right corner.

In [ ]:
plp(df, heatmap=True)

`plp` groups points together based on their proximity by setting the `cluster` argument to `True`.

In [ ]:
plp(df, cluster=True)

We can also display both the `heatmap` and `cluster` simultaneously.

In [ ]:
plp(df, heatmap=True, cluster=True)

TODO: buffer and rings

If no columns contain `lat` and `lon` keywords, or if more than two columns contain these keywords, you must explicitly specify the latitude and longitude using the `y` and `x` parameters, respectively, e.g., `plp(df, x="easting", y="northing")`. Note that `plp` assumes all data is in the [EPSG:4326](https://epsg.io/4326) projection. 

For a `GeoDataFrame`, the `plp` function can render Shapely objects such as `Point`, `LineString`, `Polygon`, and `MultiPolygon`.


Furthermore, GeoParse can visualize geospatial grids (including geohash, S2, and H3) and OSM ways (lines and polygons) directly on the map.


## Read a Polygon from Geo File

In [ ]:
gdf = gpd.read_file("https://geoparse.io/tutorials/data/london.geojson")
gdf

In [ ]:
plp(gdf)

In [ ]:
plp(gdf, centroid=True)

In [ ]:
plp(gdf, h3_res=7)

In [ ]:
plp(gdf, h3_res=10, compact=True)

In [ ]:
plp(gdf, s2_res=12)

In [ ]:
plp(gdf, s2_res=14, compact=True)

In [ ]:
plp(gdf, geohash_res=5)

In [ ]:
plp(gdf, geohash_res=7, compact=True)

# Mutiple data frames

In [ ]:
plp([df, gdf], h3_res=6)


Furthermore, GeoParse can visualize geospatial grids (including geohash, S2, and H3) and OSM ways (lines and polygons) directly on the map.


# Geospatial Cells

In [ ]:
import h3

lat, lon = 41.87, -87.78

# Get the H3 index at resolution 6 for the central point
h3_index = h3.latlng_to_cell(lat, lon, 6)

# Get adjacent H3 cells including the central cell itself
h3_cells = h3.grid_disk(h3_index, 1)  # k_ring with radius 1 returns the central cell + neighbors
h3_cells

In [ ]:
plp(cells=list(h3_cells), cell_type="h3")

# OSM Ways

In [ ]:
plp(osm_ways=[335265936, 53820456, 1117218957], s2_res=20, compact=True)

In [ ]:
plp(osm_ways=[260909736, 54454413])